## UM+X 

Test notebook to see if we can get the UM + secondary objectives implemented without a lot of hassle.

Notes on Gurobi Multi-Objective: https://www.gurobi.com/documentation/9.5/refman/specifying_multiple_object.html

### Note that the allocation code assumes that each agent can only be allocated a particular item one time (in the case of multiple copies).

In [85]:
# Includes and Standard Magic...
### Standard Magic and startup initializers.

# Load Numpy
import numpy as np
# Load MatPlotLib
import matplotlib
import matplotlib.pyplot as plt
# Load Pandas
import pandas as pd
# Load Stats
from scipy import stats
import seaborn as sns
import gurobipy as gpy
import itertools

# This lets us show plots inline and also save PDF plots if we want them
%matplotlib inline
matplotlib.style.use('fivethirtyeight')
from matplotlib.backends.backend_pdf import PdfPages

# These two things are for Pandas, it widens the notebook and lets us display data easily.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [86]:
# UM within Prop model

## Pass in a Matrix of POSITIVE BIDS and NEGATIVE for COI!

def UM_PROP_capacitated_assignment(m, pd_bid_matrix, object_caps, agent_caps, agents, objects):

    # Dicts to keep track of varibles...
    assigned = {}
    utility = {}

    #NOTE THAT THESE ARE BINARY SO WE CAN ONLY ASSIGN EACH AGENT ONCE!!
    # Create a binary variable for every agent/object.
    for a in agents:
        for o in objects:
            assigned[a,o] = m.addVar(vtype=gpy.GRB.BINARY, name='assigned_%s_%s' % (a,o))

    # Create a variable for each agent's utility.
    for a in agents:
        utility[a] = m.addVar(vtype=gpy.GRB.CONTINUOUS, name='utility_%s' % (a))

    # add the variables to the model.
    m.update()

    # Agents can't be assigned negitive objects (no preference).
    for a in agents:
        for o in objects:
            if pd_bid_matrix.loc[a,o] == -1:
                m.addConstr(assigned[a,o] == 0)

    # Enforce that items can only be allocated o times each..
    for o in objects:
        m.addConstr(gpy.quicksum(assigned[a,o] for a in agents) >= object_caps[o][0], 'object_min_cap_%s' % (o))
        m.addConstr(gpy.quicksum(assigned[a,o] for a in agents) <= object_caps[o][1], 'object_max_cap_%s' % (o))

    # Enforce that each agent can't have more than agent_cap items.
    for a in agents:
        m.addConstr(gpy.quicksum(assigned[a,o] for o in objects) >= agent_caps[a][0], 'agent_min_cap_%s' % (a))
        m.addConstr(gpy.quicksum(assigned[a,o] for o in objects) <= agent_caps[a][1], 'agent_max_cap_%s' % (a))

    # Enforce the agent utility computations.
    for a in agents:
        m.addConstr(gpy.quicksum(assigned[a,o] * pd_bid_matrix.loc[a,o] for o in objects) == utility[a], 'agent_%s_utility' % (a))

    m.update()
    
    # Add in a constraint that we have proportionality, specifically that for each agent:
    # u_i(p(i)) \geq ui(o) / n
    for a in agents:
        m.addConstr(utility[a] >= float(pd_bid_matrix.loc[a].sum()) / len(agents))
    
    # Util SW
    m.setObjective(gpy.quicksum(utility[a] for a in agents), gpy.GRB.MAXIMIZE)
    return m, assigned, utility

In [87]:

## EF Within UM Model.

## Pass in a Matrix of POSITIVE BIDS and NEGATIVE for COI!

def UM_EF_capacitated_assignment(m, pd_bid_matrix, object_caps, agent_caps, agents, objects):

    # Dicts to keep track of varibles...
    assigned = {}
    utility = {}

    #NOTE THAT THESE ARE BINARY SO WE CAN ONLY ASSIGN EACH AGENT ONCE!!
    # Create a binary variable for every agent/object.
    for a in agents:
        for o in objects:
            assigned[a,o] = m.addVar(vtype=gpy.GRB.BINARY, name='assigned_%s_%s' % (a,o))

    # Create a variable for each agent's utility.
    for a in agents:
        utility[a] = m.addVar(vtype=gpy.GRB.CONTINUOUS, name='utility_%s' % (a))

    # add the variables to the model.
    m.update()

    # Agents can't be assigned negitive objects (no preference).
    for a in agents:
        for o in objects:
            if pd_bid_matrix.loc[a,o] == -1:
                m.addConstr(assigned[a,o] == 0)

    # Enforce that items can only be allocated o times each..
    for o in objects:
        m.addConstr(gpy.quicksum(assigned[a,o] for a in agents) >= object_caps[o][0], 'object_min_cap_%s' % (o))
        m.addConstr(gpy.quicksum(assigned[a,o] for a in agents) <= object_caps[o][1], 'object_max_cap_%s' % (o))

    # Enforce that each agent can't have more than agent_cap items.
    for a in agents:
        m.addConstr(gpy.quicksum(assigned[a,o] for o in objects) >= agent_caps[a][0], 'agent_min_cap_%s' % (a))
        m.addConstr(gpy.quicksum(assigned[a,o] for o in objects) <= agent_caps[a][1], 'agent_max_cap_%s' % (a))

    # Enforce the agent utility computations.
    for a in agents:
        m.addConstr(gpy.quicksum(assigned[a,o] * pd_bid_matrix.loc[a,o] for o in objects) == utility[a], 'agent_%s_utility' % (a))

    m.update()
    
    # Add in a constraint that we have EnvyFREENESS, specifically that for each agent:
    # u_i(p(i)) >= u_i(p(j)) for all i,j in N
    for i,j in itertools.combinations(agents, 2):
        # Note we need i,j and j,i because this isn't symmetric...
        m.addConstr(utility[i] >= gpy.quicksum(assigned[j,o] * pd_bid_matrix.loc[i,o] for o in objects))
        m.addConstr(utility[j] >= gpy.quicksum(assigned[i,o] * pd_bid_matrix.loc[j,o] for o in objects))
    
    # Util SW
    m.setObjective(gpy.quicksum(utility[a] for a in agents), gpy.GRB.MAXIMIZE)
    return m, assigned, utility

In [88]:
## EF1 + UM Model.

def UM_EF1_capacitated_assignment(m, pd_bid_matrix, object_caps, agent_caps, agents, objects):

    # Dicts to keep track of varibles...
    assigned = {}
    utility = {}

    #NOTE THAT THESE ARE BINARY SO WE CAN ONLY ASSIGN EACH AGENT ONCE!!
    # Create a binary variable for every agent/object.
    for a in agents:
        for o in objects:
            assigned[a,o] = m.addVar(vtype=gpy.GRB.BINARY, name='assigned_%s_%s' % (a,o))

    # Create a variable for each agent's utility.
    for a in agents:
        utility[a] = m.addVar(vtype=gpy.GRB.CONTINUOUS, name='utility_%s' % (a))

    # add the variables to the model.
    m.update()

    # Agents can't be assigned negitive objects (no preference).
    for a in agents:
        for o in objects:
            if pd_bid_matrix.loc[a,o] == -1:
                m.addConstr(assigned[a,o] == 0)

    # Enforce that items can only be allocated o times each..
    for o in objects:
        m.addConstr(gpy.quicksum(assigned[a,o] for a in agents) >= object_caps[o][0], 'object_min_cap_%s' % (o))
        m.addConstr(gpy.quicksum(assigned[a,o] for a in agents) <= object_caps[o][1], 'object_max_cap_%s' % (o))

    # Enforce that each agent can't have more than agent_cap items.
    for a in agents:
        m.addConstr(gpy.quicksum(assigned[a,o] for o in objects) >= agent_caps[a][0], 'agent_min_cap_%s' % (a))
        m.addConstr(gpy.quicksum(assigned[a,o] for o in objects) <= agent_caps[a][1], 'agent_max_cap_%s' % (a))

    # Enforce the agent utility computations.
    for a in agents:
        m.addConstr(gpy.quicksum(assigned[a,o] * pd_bid_matrix.loc[a,o] for o in objects) == utility[a], 'agent_%s_utility' % (a))

    m.update()
    
    # Erel's idea, introduce a new y_{i,j,o} for every i,j \in A and o \in O
    # Where y_{i,j,o} is for every two agents and object o takes the highest value item that i 
    # wants in j's bundle.

    # We can then get this into another aux variable t_{i,j} to get the utility of that item
    # for agent i in j's bundle..
    y = {}
    t = {}
    
    for i,j in itertools.permutations(agents, 2):
        t[i,j] = m.addVar(vtype=gpy.GRB.CONTINUOUS, name='t_%s_%s' % (i,j))
        for o in objects:
            y[i,j,o] = m.addVar(vtype=gpy.GRB.CONTINUOUS, name='y_%s_%s_%s' % (i,j,o))
            m.update()
            # Per item Constraints.
            # There must be some item of max value.
            m.addConstr(0 <= y[i,j,o])
            # Can't be an item that is unallocated to j.
            m.addConstr(y[i,j,o] <= assigned[i,o])
        
        # Constraints over the set of items
        # There can be only one such item.
        m.addConstr(gpy.quicksum(y[i,j,o] for o in objects) <= 1)
        # Enforce t_{i,j} takes this value
        m.addConstr(t[i,j] <= gpy.quicksum(y[i,j,o] * pd_bid_matrix.loc[a,o] for o in objects))
        m.update()
        
        # Express the EF1 constraint here.
        m.addConstr(utility[i] >= gpy.quicksum(assigned[j,o] * pd_bid_matrix.loc[i,o] for o in objects) - t[i,j])
        m.update()
    
    # Util SW
    m.setObjective(gpy.quicksum(utility[a] for a in agents), gpy.GRB.MAXIMIZE)
    return m, assigned, utility


In [89]:
## UM within Prop 1 Model

def UM_PROP1_capacitated_assignment(m, pd_bid_matrix, object_caps, agent_caps, agents, objects):

    # Dicts to keep track of varibles...
    assigned = {}
    utility = {}

    #NOTE THAT THESE ARE BINARY SO WE CAN ONLY ASSIGN EACH AGENT ONCE!!
    # Create a binary variable for every agent/object.
    for a in agents:
        for o in objects:
            assigned[a,o] = m.addVar(vtype=gpy.GRB.BINARY, name='assigned_%s_%s' % (a,o))

    # Create a variable for each agent's utility.
    for a in agents:
        utility[a] = m.addVar(vtype=gpy.GRB.CONTINUOUS, name='utility_%s' % (a))

    # add the variables to the model.
    m.update()

    # Agents can't be assigned negitive objects (no preference).
    for a in agents:
        for o in objects:
            if pd_bid_matrix.loc[a,o] == -1:
                m.addConstr(assigned[a,o] == 0)

    # Enforce that items can only be allocated o times each..
    for o in objects:
        m.addConstr(gpy.quicksum(assigned[a,o] for a in agents) >= object_caps[o][0], 'object_min_cap_%s' % (o))
        m.addConstr(gpy.quicksum(assigned[a,o] for a in agents) <= object_caps[o][1], 'object_max_cap_%s' % (o))

    # Enforce that each agent can't have more than agent_cap items.
    for a in agents:
        m.addConstr(gpy.quicksum(assigned[a,o] for o in objects) >= agent_caps[a][0], 'agent_min_cap_%s' % (a))
        m.addConstr(gpy.quicksum(assigned[a,o] for o in objects) <= agent_caps[a][1], 'agent_max_cap_%s' % (a))

    # Enforce the agent utility computations.
    for a in agents:
        m.addConstr(gpy.quicksum(assigned[a,o] * pd_bid_matrix.loc[a,o] for o in objects) == utility[a], 'agent_%s_utility' % (a))

    m.update()
    
    # Introduce a new n_{i,o} for every i and o in O
    # Where n_{i,o} is the highest value unallocated item to this agent.

    # We can then get this into another aux variable t_{i} to get the utility of that item to agent i
    n = {}
    t = {}
    
    for i in agents:
        t[i] = m.addVar(vtype=gpy.GRB.CONTINUOUS, name='t_%s' % (i))
        for o in objects:
            n[i,o] = m.addVar(vtype=gpy.GRB.BINARY, name='n_%s_%s' % (i,o))
            m.update()
            # Per item Constraints.
            # Can't be an item that is already allocated to i
            m.addConstr(n[i,o] <= 1 - assigned[i,o])
            m.update()
        
        # Constraints over the set of items
        # There can be only one such item.
        m.addConstr(gpy.quicksum(n[i,o] for o in objects) <= 1)
        # Enforce t_{i} takes the utility of this max value object..
        m.addConstr(t[i] <= gpy.quicksum(n[i,o] * pd_bid_matrix.loc[i,o] for o in objects))
        m.update()
        
        # Express Prop1 Constraint here.
        m.addConstr(utility[i] + t[i] >= float(pd_bid_matrix.loc[i].sum()) / len(agents))
        m.update()
    
    # Util SW
    m.setObjective(gpy.quicksum(utility[a] for a in agents), gpy.GRB.MAXIMIZE)
    m.write("./test.lp")
    return m, assigned, utility

In [90]:
# Code is expecting a REVIEWER x ITEM Matrix..

# data = {"Agent1": {"Obj1": 1, "Obj2": 2, "Obj3": 3},
#         "Agent2": {"Obj1": 1, "Obj2": 2, "Obj3": 3},
#         "Agent3": {"Obj1": 1, "Obj2": 2, "Obj3": 3},
#        }


# EF1 but not EF

data = {"Agent1": {"Obj1": 2, "Obj2": 1, "Obj3": 1, "Obj4": 6},
        "Agent2": {"Obj1": 1, "Obj2": 1, "Obj3": 3, "Obj4": 5}
       }

# # Prop1 but not Prop...

# data = {"Agent1": {"Obj1": 4, "Obj2": 1, "Obj3": 1, "Obj4": 1, "Obj5": 1, "Obj6": 1, "Obj7": 1},
#         "Agent2": {"Obj1": 4, "Obj2": 1, "Obj3": 1, "Obj4": 1, "Obj5": 1, "Obj6": 1, "Obj7": 1}
#        }


df_test_bid = pd.DataFrame.from_dict(data, orient="index")
df_test_bid

,Obj1,Obj2,Obj3,Obj4
Agent1,2,1,1,6
Agent2,1,1,3,5


In [91]:
# Call the allocation function with this set...
# A simple call...
agents = list(df_test_bid.index)
objects = list(df_test_bid.columns)

# Global set to 1/1 can be whatever... min/max
object_caps = {i:(1,1) for i in objects}
agent_caps = {i:(1,10) for i in agents}

m = gpy.Model('Util')

m, assigned, utility = UM_PROP1_capacitated_assignment(m, df_test_bid, object_caps, agent_caps, agents, objects)
m.setParam(gpy.GRB.Param.OutputFlag, False )
m.optimize()
print("Finished in (seconds): " + str(m.Runtime))

if m.Status == gpy.GRB.OPTIMAL:
    print("Objective Value: " + str(m.ObjVal))
else:
    print("No Solution")



Finished in (seconds): 0.0014760494232177734
Objective Value: 12.0


In [92]:
assignment = {}
for a in agents:
    assignment[a] = [o for o in objects if assigned[a,o].x > 0.1]
assignment

{'Agent1': ['Obj1', 'Obj2', 'Obj4'], 'Agent2': ['Obj3']}